<a href="https://colab.research.google.com/github/Debby190/text-processing/blob/main/femaledaily1_kelompok2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from bs4 import BeautifulSoup
import requests as rs
import pandas as pd
import re
import string

# **start**

In [7]:
link = 'https://reviews.femaledaily.com/products/eyes/mascara/maybelline/volum-express-hyper-curl-54?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page='
page_response = rs.get(link)
page_response

<Response [200]>

In [8]:
soup = BeautifulSoup(page_response.text, 'html.parser')
rev = soup.find('p', {'class': 'text-content'}).text
print (rev)
rekom = soup.find('p', {'class':"recommend"}).text
print (rekom)
tanggal = soup.find('p', {'class':"review-date"}).text
print (tanggal)
reviewer = soup.find('p', {'class':"profile-username"}).text
print (reviewer)


Menurut ku maskara ini bikin lentik bulu mata dan keliatan lebih bervolume, aplikatornya juga memudahkan buat nyisir bulu matanya. Packagingnya juga simple tpi bisa hold product nya dengan baik. Cuma sayang karna isinya terlalu banyak dan jadi bleber kalo jarang dipake
yohananilamsari recommends this product!
4 days ago
yohananilamsari


In [9]:
review = []
tanggal = []
reviewer = []
root = 'https://reviews.femaledaily.com/products/eyes/mascara/maybelline/volum-express-hyper-curl-54?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page='


In [10]:

for page in range(1, 100):
    url = rs.get(root + str(page))
    soup = BeautifulSoup(url.text, 'html.parser')

    def scraping(soup):
        ulasan = soup.findAll('div', {'class': 'review-card'})

        for i in ulasan:
            rev = i.find("p", {"class": "text-content"})
            tgl = i.find("p", {"class": "review-date"})
            nama = i.find("p", {"class": "profile-username"})
            if tgl:
                tgl_text = tgl.text.strip()
                if re.search(r'(Jun|Jul|Aug|Sep)\s+2024', tgl_text):
                  tanggal.append(tgl.text.strip() if tgl else None)
                  review.append(rev.text.strip() if rev else None)
                  reviewer.append(nama.text.strip() if nama else None)

    scraping(soup)

In [11]:
df = pd.DataFrame({
    'Tanggal': tanggal,
    'Reviewer': reviewer,
    'Review': review
})

In [12]:
df = df.reindex(columns=['Tanggal', 'Reviewer', 'Review'])
df.to_csv("review_produk.csv", index=False, encoding='utf-8-sig')
print(f'Data berhasil disimpan ke "review_produk.csv". Total halaman yang di-scrap: {page - 1}')

Data berhasil disimpan ke "review_produk.csv". Total halaman yang di-scrap: 98


In [13]:
def preprocess_lowercase(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].str.lower()  # Mengubah teks di kolom menjadi lowercase
        else:
            print(f"Kolom {col} tidak ditemukan di DataFrame.")
    return df



In [14]:
def remove_emoticons(text):
    emoticon_pattern = r'[:;=][oO\-]?[\)DdpP3/\\|o\[\]{}<>]'
    return re.sub(emoticon_pattern, '', text)

In [15]:
def preprocess_remove_emoticons(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(remove_emoticons)  # Menghapus emotikon di kolom asli
        else:
            print(f"Kolom {col} tidak ditemukan di DataFrame.")
    return df

In [16]:
def remove_punctuation(text):
    if isinstance(text, str):
        return re.sub(r'[^a-z\s]', '', text)  # Menghapus semua karakter kecuali huruf, angka, dan spasi
    return text

In [17]:
def preprocess_remove_punctuation(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(remove_punctuation)  # Menghapus punctuation di kolom asli
        else:
            print(f"Kolom {col} tidak ditemukan di DataFrame.")
    return df


In [18]:
def remove_stopwords(text):
    stopwords = ['di', 'ke', 'dari', 'pada', 'untuk', 'dengan', 'adalah', 'yang', 'dan', 'atau',
             'jika', 'itu', 'ini', 'tetapi', 'karena','juga', 'akan', 'telah', 'dalam', 'bnr2',
             'soal', 'semua', 'aja', 'doang', 'sama', 'banget', 'apalagi', 'emang',
             'kalo', 'jadi', 'untuk', 'tuh', 'makanya', 'dulu', 'masih', 'pas','sih', 'gitu',
             'menurut', 'ya', 'bikin', 'buat', 'ada']
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

df['Review'] = df['Review'].apply(remove_stopwords)

In [19]:
df = pd.DataFrame({
    'Tanggal': tanggal,
    'Reviewer': reviewer,
    'Review': review
})

In [20]:
df = preprocess_lowercase(df, ['Reviewer', 'Review'])
df = preprocess_remove_emoticons(df, ['Review'])
df = preprocess_remove_punctuation(df, ['Review'])

In [21]:
print(df.head)

<bound method NDFrame.head of          Tanggal       Reviewer  \
0    30 Sep 2024        saroh10   
1    30 Sep 2024    sekarrmln24   
2    30 Sep 2024   niethakopa85   
3    29 Sep 2024          shea6   
4    29 Sep 2024     runilaanwr   
..           ...            ...   
421  04 Jun 2024  izzasalsabila   
422  04 Jun 2024     floryartus   
423  04 Jun 2024  mawarsyafitri   
424  03 Jun 2024    shakiladian   
425  02 Jun 2024    ratrigandis   

                                                Review  
0    ini mascaraaaaaa kecintaaan gue bangett sumpaa...  
1    salah satu mascara fav apalagi buat daily kare...  
2    ini tuh maskara pertama yang aku pakai dari pa...  
3    dulu pakai ini pas masih smabuat anak sma masi...  
4    bulumata ku tipe yang lurus terus agak kaku be...  
..                                                 ...  
421  beuhh baru dipake sejam doang bawah mata udah ...  
422  bagus bgt worth to buy harganya juga murah ga ...  
423  maskara yang menambah volume bu

In [22]:
df.to_csv("review_processed.csv", index=False, encoding='utf-8-sig')

In [23]:
!pip install nltk

In [30]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = {
    'slang': ['ga', 'gak', 'aku', 'gue', 'tp', 'lo', 'banget', 'krn', 'tks', 'yg'],
    'formal': ['tidak', 'tidak', 'saya', 'saya', 'tapi', 'kamu', 'sangat', 'karena', 'terima kasih', 'yang']
}

In [31]:
df_slang = pd.DataFrame(data)
df_slang.to_csv('colloquial_indonesian_lexicon.csv', index=False, encoding='utf-8-sig')
print("File CSV kamus slang telah dibuat!")

File CSV kamus slang telah dibuat!


In [25]:
df_slang = pd.read_csv('colloquial_indonesian_lexicon.csv')

In [32]:
dict_alay = df_slang.set_index('slang')['formal'].to_dict()

In [33]:
def normalize_text(data):
    try:
        word_tokens = word_tokenize(data)
    except Exception as e:
        print(f"Error in tokenization: {e}")
        return data
    result = [dict_alay.get(wt, wt) for wt in word_tokens]
    return ' '.join(result)


In [34]:
df_reviews = pd.read_csv('review_processed.csv')
df_reviews['Review_Normalized'] = df_reviews['Review'].apply(normalize_text)
df_reviews.to_csv("review_produk_normalized.csv", index=False, encoding='utf-8-sig')
print("Data review berhasil dinormalisasi dan disimpan ke 'review_produk_normalized.csv'.")

Data review berhasil dinormalisasi dan disimpan ke 'review_produk_normalized.csv'.


FEATURE ENGINEERING


In [54]:
!pip install Sastrawi
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 19.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=c4d29b16019f3d65adfaad71e6405d109f2e9289f123b8eb6add8fd17d883876
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [57]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(data):
    return stemmer.stem(data)

review_dict = {}

for document in ['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            review_dict[review] = ' '

print(len(review_dict))
print("------------------------")

for review in review_dict:
    review_dict[review] = stemmed_wrapper(review_dict)
    print(term,":" ,review_dict[review])

print(review_dict)
print("------------------------")


# # apply stemmed term to dataframe
# def get_stemmed_term(document):
#     return [review_dict[Review] for review in document]

# TWEET_DATA['review_produk_stemming'] = TWEET_DATA['Review_Normalized'].swifter.apply(get_stemmed_term)
# print(TWEET_DATA['tweet_tokens_stemmed'])

NameError: name 'term_dict' is not defined

In [46]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

df_reviews = pd.read_csv('review_produk_normalized.csv')
df_reviews['Review_Normalized'] = df_reviews['Review'].apply(stemming_text)
df_reviews.to_csv("review_produk_stemming.csv", index=False, encoding='utf-8-sig')
print("Data review berhasil dinormalisasi dan disimpan ke 'review_produk_stemming.csv'.")


KeyboardInterrupt: 

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
corpus_file ="review_produk_normalized.csv"
output_file = "TFIDF_review.csv"

data = pd.read_csv(corpus_file)  # Misalnya, jika file dipisahkan dengan titik koma
data.head()


,Tanggal,Reviewer,Review,Review_Normalized
0,30 Sep 2024,saroh10,ini mascaraaaaaa kecintaaan gue bangett sumpaa...,ini mascaraaaaaa kecintaaan saya bangett sumpa...
1,30 Sep 2024,sekarrmln24,salah satu mascara fav apalagi buat daily kare...,salah satu mascara fav apalagi buat daily kare...
2,30 Sep 2024,niethakopa85,ini tuh maskara pertama yang aku pakai dari pa...,ini tuh maskara pertama yang saya pakai dari p...
3,29 Sep 2024,shea6,dulu pakai ini pas masih smabuat anak sma masi...,dulu pakai ini pas masih smabuat anak sma masi...
4,29 Sep 2024,runilaanwr,bulumata ku tipe yang lurus terus agak kaku be...,bulumata ku tipe yang lurus terus agak kaku be...


In [37]:
corpus = data['Review'].astype(str).tolist()

# TF-IDF
vectorizer = TfidfVectorizer()
tfidf_encoded = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()

# Mengonversi hasil TF-IDF menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_encoded.toarray(), columns=feature_names)

tfidf_df.to_csv(output_file, index=False)

print("TF-IDF selesai. Hasilnya telah disimpan di:", output_file)
display(tfidf_df)

TF-IDF selesai. Hasilnya telah disimpan di: TFIDF_review.csv


,aa,aaan,abaaaallll,abis,abisabis,abisin,abisnya,abiss,about,absolutely,...,year,yey,yg,york,you,your,yuk,yukk,zaman,zuzur
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.173831,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.102308,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.101953,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
